# Self Self Catalysis

1. Find alpha =/> beta and beta =/> alpha pairs
2. Calculate alpha x alpha (alpha') and beta x alpha(beta')
3. Sort alpha' and beta'
4. Predict/Check if alpha' -> beta'
5. Find alpha' =/> beta' and beta' =/> alpha' pairs
6. Calculate alpha' x alpha (alpha'') and beta' x alpha(beta'')
7. Sort alpha'' and beta''
8. Check if alpha'' -> beta''

In [ ]:
import time

## Data preprocessing

In [ ]:
%run ../ml_models/CommonFunctions.ipynb

In [ ]:
# dim = 4
# optimizer = 'adam'
# predict_flag = True

In [ ]:
df_sample = read_csv('../data/output_d'+str(dim)+'_s1000000.csv')
df_input = format_input(df_sample)

In [ ]:
initial_time = time.time()

## Filtering not majorated samples

In [ ]:
df_not_majorated = df_input[df_input['AB'] == False]
#TODO filter only b =/> a to get stronger results

In [ ]:
df_not_majorated.head()

## Loading model and weights

In [ ]:
if predict_flag:
    self_model = load_model(dim**2, optimizer)
    def predict(A, B):
        return False if self_model.predict([np.array([A]), np.array([B])]) < 0.5 else True

## Multiplying and sorting elements

In [ ]:
def self_catalysis_multiply_and_sort(row):
    newA = np.sort(np.multiply.outer(row["A"], row["A"]).flatten())[::-1]
    row['Al'] = newA
    newB = np.sort(np.multiply.outer(row["B"], row["A"]).flatten())[::-1]
    row['Bl'] = newB
    row['ABl'] = predict(newA, newB) if predict_flag else is_a_valid_pair(newA, newB)
    return row

df_self_catalysis = df_not_majorated.reset_index(drop=True).apply(self_catalysis_multiply_and_sort, axis=1)

In [ ]:
df_self_catalysis.head()

In [ ]:
df_self_not_majorated = df_self_catalysis[df_self_catalysis['ABl'] == False]

In [ ]:
df_self_not_majorated.head()

In [ ]:
def self_self_catalysis_multiply_and_sort(row):
    newA = np.sort(np.multiply.outer(row["Al"], row["A"]).flatten())[::-1]
    row['All'] = newA
    newB = np.sort(np.multiply.outer(row["Bl"], row["A"]).flatten())[::-1]
    row['Bll'] = newB
    row['ABll'] = is_a_valid_pair(newA, newB)
    return row

df_self_self_catalysis = df_self_not_majorated.reset_index(drop=True).apply(self_self_catalysis_multiply_and_sort, axis=1)

In [ ]:
final_model = load_model(dim**3, optimizer)

In [ ]:
start_time = time.time()
score = final_model.evaluate([np.array(list(df_self_self_catalysis['All'])), np.array(list(df_self_self_catalysis['Bll']))], np.array(list(df_self_self_catalysis['ABll'])))
catalysis_time = time.time() - start_time
complete_time = time.time() - initial_time

In [ ]:
score

In [ ]:
output_name = 'results/self_self_catalysis_acc_predict.csv' if predict_flag else 'results/self_self_catalysis_acc.csv'
save_data(output_name,optimizer,dim,score[1])

In [ ]:
output_name = 'results/self_self_catalysis_execution_time_predict.csv' if predict_flag else 'results/self_self_catalysis_execution_time.csv'
save_data(output_name,optimizer,dim,catalysis_time)

In [ ]:
output_name = 'results/self_self_catalysis_complete_time_predict.csv' if predict_flag else 'results/self_self_catalysis_complete_time.csv'
save_data(output_name,optimizer,dim,complete_time)